# Nginx配置

## 参考文献

* *深入理解Nginx，模块开发与架构解析*
* [nginx官方文档](http://nginx.org/en/docs/)
* *https://zhuanlan.zhihu.com/p/51251700*

## 编译安装Nginx

```shell
cd ~/github/nginx-1.4;
./auto/configure --prefix=/Users/hotbaby/github/nginx-1.4/ngin;

make -j8;
make isntall;
```


## Nginx进程间的关系

Nginx使用一个Master进程管理多个worker进程，一般情况下，worker进程的数量和服务器上CPU核心数相等。

* master进程不会对用户请求提供服务，只用于管理真正提供服务的worker进程，包括启动服务、停止服务、重载配置文件，平滑升级等；
* 多个worker进程处理互联网请求不但可以提供服务的健壮性，还可以利用现在SMP多核架构。

## Nginx配置优化

### 优化性能

Nginx worker进程数量

*  `worker_process number`
* `worker_cpu_affinity cpumask [cpumask ...]`

1. 多worker进程可以充分利用多核架构，但若worker进程的数量超过CPU核心数，那么会增大进程间切换带来的消耗，一般情况下worker进程与CPU核心数量相等
2. 绑定Nginx worker进程到指定的CPU内核

**为什么要绑定worker进程到指定的CPU内核？**

假定每个worker进程都非常繁忙，如果多个进程都会抢占同一个CPU，那么就会出现同步问题。反之，如果每个worker进程都独享一个CPU，就在内核调度策略上实现了完全的并发。

### 事件类配置

`accept_mutex  [on | off]`

`accept_mutex`可以让多个worker进程轮流、序列化与新的客户端建立TCP连接。当一个worker进程建立的连接数达到`worker_connections`配置最大连接数的7/8时，会大大地减少该worker进程试图建立TCP连接到机会，以此实现所有worker进程上处理客户端请求数尽量接近。

accept锁默认是打开的，如果关闭它，那么建立TCP连接到耗时会更短，但worker进程负载会非常不均衡。

选择事件模型

`use [select | poll | epoll | kqueue]`

每个worker的最大连接数

`worker_connections number`

## 配置静态Web服务器

### 虚拟主机与请求的分发

监听端口

`listen address:port default | default_server | backlog=num | deffered`

**`default` 将所在的server模块作为整个Web服务器默认的server模块。如果没有设置这个参数，那么将会以`nginx.conf`中找到的第一个server块作为默认的server块。**

> Nginx实践中，最好配置`default_server`，避免未匹配的请求打到不期望的的`server`上。

**为什么需要默认默认虚拟主机呢？**

当一个请求无法匹配配置文件中所有主机域时，就会选用默认的虚拟主机。

`backlog=num`

表示TCP中backlog队列的大小。默认为-1，表示不予设置。在TCP建立三次握手过程中，进程还没有开始处理监听的句柄，就会将这连接放在backlog队列中。如果backlog队列已满，还有新的的客户端试图通过三次握手建立TCP连接，就会失败。

`deferred`

设置该参数候，若用户发起建立连接请求，并且完成了TCP的三次握手，内核也不会为这次的连接调度worker进程来处理，只有用户真的发送请求数据，内存才会唤醒worker进程处理这个连接。这个参数使用大并发的情况下，它减轻了worker进程都负担。

主机名称

`server_name name [...]`

`server_name`后可以跟多个主机名称，例如`server_name www.bytehandler.com download.bytehandler.com`

在开始处理一个HTTP请求时，Nginx会取出header的Host，与每个server的`server_name`进行匹配，以此决定使用哪个`server`块来处理这个请求。存在多个server块配置，根据优先级匹配，server_name与Host匹配优先级如下：

1. 所有字符串完全匹配的`server_name`
2. 通配符在前面的`server_name`，例如：`*.bytehandler.com`
3. 通配符在后面的`server_name`，例如：`www.bytehandler.*`
4. 最后选择使用正则表达式才能匹配的`server_name`，例如：`~^\.bytehandler\.com`

> Nginx使用`server_name`配置针对特定的Host匹配不同的服务，以此实现虚拟主机功能。

`server_names_hash_bucket_size size;`

为了提高快速寻找相应的`server_name`的能力，Nginx使用散列表存储`server_name`，`server_names_hash_bucket_size`设置了每个散列桶占用内存的大小。

`server_names_hash_max_size size;`

`server_names_hash_max_size`会影响散列表的冲突率。`server_names_hash_max_size`越大，消耗的内存越大，散列key冲突的概率越低，检索的效率也会更快。

`location [=|~|~*|^~|@] uri/ {}`

`location`会尝试根据用户的URI来匹配上面的URI表达式。

* = 表示完全匹配
* ~ 表示匹配URI时是字母大小写敏感的
* ~* 表示匹配URI时，忽略字母的大小问题
* ^~ 表示匹配URI时，只需要前半部分与URI参数匹配即可

网络连接设置

keepalive超时时间

`keepalive_timeout time(默认单位：秒)`

一个keepalive连接在闲置超过一定的时间后（默认75秒），服务器和浏览都会去关闭这个连接。当然keepalive_timeout是用来约束Nginx服务器的，Nginx也会按照规范讲个这个时间传给浏览器。

`tcp_nodely on|off`

确定对`keepalive`连接是否使用`TCP_NODELY`选项。

## 用HTTP proxy module配置一个反向代理服务器

反向代理（reverse proxy）是指用户代理服务器接收Internet的连接请求，然后将请求转发给内部网络的上游服务器，并将上游服务器得到的结果返回给Internet上连接的客户端。

`upstream name {}`

```nginx
upstream backend {
    server backend1.bytehander.com;
    server backend2.bytehandler.com；
}
```

`server name [parametes]`

* `weigth=number`，设置这台服务器转发的权重，默认为1
* `max_fails=number`，该参数与`fail_timeout`配和使用，如果上游服务器转发失败次数超过number，则认为fail_timeout时间段内的上游服务器不可用。
* `fail_timeout=time`
* `down`，表示所有在上游服务器永久下线
* `backup`，在使用ip_hash配置时才有效，表示所在上游服务器只是备份服务器，只有所有非备份服务器上游服务器都失败后，才会向所在上游服务器转发请求

`ip_hash` 

在有些场景下，希望某一个用户的请求使用落在固定的一台服务器中。

`ip_hash`与`weight`配置不可同时使用。如果`upstream`集群中有一台上游服务器暂时不可用，不能直接删除配置，而是要配置`down`参数标识。

## 应用

### Location重定向

参考文献：

* http://nginx.org/en/docs/http/ngx_http_rewrite_module.html
* https://www.cnblogs.com/tugenhua0707/p/10798762.html

```nginx
upstream upstream_name {
    server ip:port;
}

server {
    listen 80;
    server_name bytehandler.com;
    client_max_body_size 300M;

    location =/ {
        # 重定向
        rewrite / /URI redirect;
    }

    location / {
        proxy_pass http://upstream_name;
        proxy_http_version 1.1;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection upgrade;
    }
}
```

### 防盗链

参考文献：https://www.cnblogs.com/limeng951/p/5833790.html

```nginx
location ~* \.(gif|jpg|jpeg|png|swf|flv)$ { 
    valid_referers none blocked *.bytehandler.com ; 
    if ($invalid_referer) { 
        #rewrite ^/ http://www.bytehandler.com/; 
        return 403; 
    } 
}
```

### 自动索引目录

```nginx
server {
    listen 80;
    server_name pic.bytehandler.com;

    location / {
        # 防盗链
        valid_referers none blocked pic.bytehandler.com ;
        if ($invalid_referer) {
            return 403;
        }

        # autoindex
        alias /home/hotbaby/Pictures/;
        autoindex on;
        autoindex_exact_size off;
        charset utf-8;
        autoindex_localtime on;
    }

}
```

* 配置了防盗链，跨域不能打开图片是正常现象
* `charset utf-8;`解决中文乱码问题

![配置了防盗链，因此打开不图片是正常现象](http://pic.bytehandler.com/孟阳阳.jpeg)